In [12]:
#!pip install ibapi

In [13]:
import time
import threading
from datetime import datetime
from typing import Dict, Optional
import pandas as pd
import warnings

In [14]:
from ibapi.client import EClient # makes calls 
from ibapi.wrapper import EWrapper #    receives responses
from ibapi.contract import Contract # contract instrument       
from ibapi.order import Order # order object
from ibapi.common import BarData # historical data bars


In [ ]:
#make a class that inherits from EClient and EWrapper
class TradingApp (EClient, EWrapper):
    def __init__(self):  
        EClient.__init__(self, self)
        self.data:Dict[int, pd.DataFrame] = {}

    def error(self, reqId: int, errorcode: int, errorstring: str, advanced: any) -> None:
        print("Error: ", reqId, " ", errorcode, " ", errorstring)

    def get_historical_data(self, reqId: int, contract: Contract) -> pd.DataFrame:
        self.data[reqId] = pd.DataFrame(columns=['time', 'high', 'low', 'close'])
        self.data[reqId].set_index('time', inplace=True)
        self.reqHistoricalData(
            reqId = reqId,
            contract = contract,
            endDateTime='',
            durationStr='1 D',
            barSizeSetting='1 min',
            whatToShow='MIDPOINT',
            useRTH=0,
            formatDate=2,
            keepUpToDate=False,
            chartOptions=[]
        )
        time.sleep(3)
        return self.data[reqId]
    
    def historicalData(self, reqId: int, bar: BarData) -> None:
        df = self.data[reqId]
        df.loc[
            pd.to_datetime(bar.date, unit = 's'),
            ['high', 'low', 'close']
        ] = [bar.high, bar.low, bar.close]
        df = df.astype(float)
        self.data[reqId] = df

    @staticmethod
    def get_stock_contract(symbol: str) -> Contract:
        contract = Contract()
        contract.symbol = symbol
        contract.secType = 'STK'
        contract.exchange = 'SMART'
        contract.currency = 'USD'
        return contract
  

In [17]:
app = TradingApp()

In [ ]:
app.connect()